# Hurricane Data to SQL database


load necessary packages

In [ ]:
import sqlite3
import pandas as pd

load necessary dataset

In [ ]:
hrcn_risk_data = pd.read_csv('mainland_usa_gdf_HRCN.csv', dtype={'3ZIP':str})
hrcn_risk_data = hrcn_risk_data[['3ZIP', 'HRCN_RISKS', 'HRCN_RISKV', 'HRCN_EVNTS', 'HRCN_EALS']]

In [ ]:
#Save to accessible pickle file


## Open connection SQL

In [ ]:
db_path = "../Database/thesis_database.db"
conn = sqlite3.connect(db_path)

### Transform dataset to necessary

In [ ]:
hrcn_risk_data.to_sql('hrcn_risk_data', conn, if_exists = "replace", index = False)

## Close connection SQL

In [ ]:
conn.close()